# Tutorial 1
In this tutorial, we are going to build an SNN capable of classifying MNIST by copying the weights obtained by training the following simple ANN using TensorFlow:

Using GeNN for spike-based machine learning.svg

Clearly, this is far from a state of the art architecture, but it still achieves 97.6% accuracy on MNIST.

## Install
Download wheel file

In [1]:
if "google.colab" in str(get_ipython()):
    !gdown 1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
    !pip install pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
    %env CUDA_PATH=/usr/local/cuda

    !rm -rf /content/ml_genn-ml_genn_2_2_1
    !wget https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_2_1.zip
    !unzip -q ml_genn_2_2_1.zip
    !pip install ./ml_genn-ml_genn_2_2_1/ml_genn

Downloading...
From: https://drive.google.com/uc?id=1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
To: /content/pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
100% 8.38M/8.38M [00:00<00:00, 143MB/s]
Processing ./pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
pygenn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
env: CUDA_PATH=/usr/local/cuda
--2024-10-14 17:04:16--  https://github.com/genn-team/ml_genn/archive/refs/tags/ml_genn_2_2_1.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_2_1 [following]
--2024-10-14 17:04:17--  https://codeload.github.com/genn-team/ml_genn/zip/refs/tags/ml_genn_2_2_1
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9

## Download pre-trained weights

In [2]:
!gdown 1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
!gdown 131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF

Downloading...
From: https://drive.google.com/uc?id=1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
To: /content/weights_0_1.npy
100% 402k/402k [00:00<00:00, 87.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
To: /content/weights_1_2.npy
100% 5.25k/5.25k [00:00<00:00, 16.9MB/s]


## Install MNIST package

In [3]:
!pip install mnist

## Build model
Import standard modules and required mlGeNN classes

In [4]:
import mnist
import numpy as np

from ml_genn import InputLayer, Layer, SequentialNetwork
from ml_genn.compilers import InferenceCompiler
from ml_genn.neurons import IntegrateFire, IntegrateFireInput
from ml_genn.connectivity import Dense
from ml_genn.callbacks import SpikeRecorder

Because our network is entirely feedforward, we can define it as a ``SequentialNetwork`` where each layer is automatically connected to the previous layer. We are going to convert MNIST digits to spikes by simply treating the intensity of each pixel (multiplied by a scaling factor) as the input to an integrate-and-fire neuron. For our hidden and output layers we are going use very simple Integrate-and-Fire neurons which best match the transfer function of the ReLU neurons our ANN was trained with. Finally, we are going to read classifications from our output layer by simply counting spikes.



In [5]:
# Create sequential model
network = SequentialNetwork()
with network:
    input = InputLayer(IntegrateFireInput(v_thresh=5.0), 784)
    Layer(Dense(weight=np.load("weights_0_1.npy")),
          IntegrateFire(v_thresh=5.0))
    output = Layer(Dense(weight=np.load("weights_1_2.npy")),
                   IntegrateFire(v_thresh=5.0, readout="spike_count"))

In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, as we are performing inference, we use the InferenceCompiler and specify batch size and how many timesteps to evaluate each example for.

In [6]:
compiler = InferenceCompiler(dt=1.0, batch_size=128,
                             evaluate_timesteps=100)
compiled_net = compiler.compile(network)

We can then load the MNIST testing data and evaluate the compiled pre-trained network on it. The normal grayscale values of the MNIST dataset are passed into the input layer, scaled by a factor (here ```0.01```) which is translated into spikes by the ```IntegrateFireInput``` layer. The testing labels are passed as a simple array of of ```0``` to ```9```, corresponding to the default ```SparseCategorical``` loss type.

In [7]:
# Load testing
mnist.datasets_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"
testing_images = np.reshape(mnist.test_images(), (-1, 784))
testing_labels = mnist.test_labels()

with compiled_net:
    # Evaluate model on numpy dataset
    metrics, _ = compiled_net.evaluate({input: testing_images * 0.01},
                                       {output: testing_labels})


  0%|          | 0/79 [00:00<?, ?it/s]